## 12.5 Wstrzykiwanie wartości (Value Injection)

Zajmijmy się teraz przekazaniem wartości przez konstruktor w czasie wykonania. Naszym przykładem dalej będzie `Computer`, tym razem będzie posiadał tylko jeden komponent - `GPU`. Klasa `GPU` będzie przyjmować jeden paramtetr przez konstruktor - liczbę rdzeni typu `Int`.

In [ ]:
class Computer @Inject constructor(private val gpu: GPU) {
    fun getGpuCores(): String{
        return gpu.gpuCores.toString()
    }
}

class GPU(val gpuCores: Int) {}

Teraz nie możemy wykorzystać adnotacji `@Inject` przy konstruktorze klasy `GPU` - liczba rdzeni będzie znana w czasie wykonania aplikacji (może być podana przez użytkownika). Musimy manualnie wywołać konstruktor i w jakiś sposób przekazać wartość `gpuCores`.

Tym razem nasza klasa z adnotacją `@Module` będzie posiadała konstruktor w którym przekażemy wartość `gpuCores`

In [ ]:
@Module
class GpuModule(private val gpuCores: Int) {

    @Provides
    fun provideGpu(): GPU{
        return GPU(gpuCores)
    }
}

Naszx `ComputerComponent` nie zawiera żadnych nowych elementów.

In [ ]:
@Component(modules = [GpuModule::class])
interface ComputerComponent {
    fun inject(mainActivity: MainActivity)
}

Zmiany natomiast pojawią się w automatycznie wygenerowanej klasie `DaggerComputerComponent`. Nie mamy dostępnej metody `create`, teraz musimy manualnie wywołać konstruktor klasy `GpuModule`. Do tego automatycznie generowana jest implementacja wzorca projektowego [budowniczy](https://refactoring.guru/pl/design-patterns/builder).

In [ ]:
val component = DaggerComputerComponent.builder()
    .gpuModule(GpuModule(8)) // możemy przekazać liczbę rdzeni w czasie wykonania
    .build()

Tak jak w poprzednich przykładach, trworzymy instancję obiektu o typie interfejsu `ComputerComponent`, następnie wywołujemy metodę `inject`

In [ ]:
class MainActivity : AppCompatActivity() {

    @Inject
    lateinit var computer: Computer

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val component = DaggerComputerComponent.builder()
            .gpuModule(GpuModule(8))
            .build()

        component.inject(this)
        val textView = findViewById<TextView>(R.id.textview)
        textView.text = computer.getGpuCores()
    }
}

### Metoda dostarczająca

W klasie `GpuModule` przekazujemy jawnie wartość `gpuCores`, możemy jednak zaimplementować metodę dostarczającą dla tej zmiennej - dzięki temu możemy tą jedną wartość dostarczyć do wielu obiektów - jednym z zastosowań w androidzie jest przekazanie **applicationContext**.

Powróćmy do klasy `GpuModule` i dodajmy metodę `provideGpuCores`

In [ ]:
@Provides
fun provideGpuCores(): Int{
    return gpuCores
}

Teraz możemy wstrzyknąć za pomocą konstruktora `GPU`

In [ ]:
class GPU @Inject constructor (val gpuCores: Int) {}

Teraz z kolei, możemy zmodyfikować metodę `provideGpu` klasy `GpuModule` i przekazać `gpu` przez parametr.

In [ ]:
@Provides
fun provideGpu(gpu: GPU): GPU{
    return gpu
}

Tutaj konieczne jest  pamiętanie i jednej istotnej rzeczy - `Dagger2` nie interesuje nazwa zmiennej, tylko jej **typ**. Co może być kłopotliwe - jeżeli konstruktor przyjmuje dwa parametry o typie `Int`, `Dagger2` ich nie rozróżni. Jest to możliwe z kilkoma dodatkowymi adnotacjami. Elementy przedstawione do tej pory są wystarczające nac potrzeby tych zajęć, więc tutaj zakończymy omawianie biblioteki `Dagger2`.